In [1]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'composite_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'cpu', 'random_seed': 4567, 'data_configs': {'data_class': 'tinybig.data.composite_function', 'data_parameters': {'name': 'elementary_function', 'train_batch_size': 100, 'test_batch_size': 100, 'equation_index': 3}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 2000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.003, 'weight_decay': 0.0001}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.999}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'cpu', 'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [2, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_lay

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  cpu ; random_seed:  4567


In [3]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]

print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))


instantiating objects from config...
parameter num:  2441


In [4]:
print('loading dataset...')
data_loader = data_obj.load()
print(data_loader['str_equation'])

loading dataset...
C.3,3,f,exp(x+y)+ln(x+y),2,x,0,1,y,0,1


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device,
                                     metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 10.326179218292236, Test Score: 10.326179241023178, Time Cost: 0.009999990463256836
Epoch: 100, Test Loss: 7.914843590697273e-05, Test Score: 7.914843388686176e-05, Time Cost: 0.010702133178710938
Epoch: 200, Test Loss: 3.235460844734917e-05, Test Score: 3.2354608626100455e-05, Time Cost: 0.010865926742553711
Epoch: 300, Test Loss: 2.7068096096627413e-05, Test Score: 2.7068096045027135e-05, Time Cost: 0.011493921279907227
Epoch: 400, Test Loss: 1.9416726900089998e-05, Test Score: 1.9416726816243402e-05, Time Cost: 0.009797811508178711
Epoch: 500, Test Loss: 2.3604950797562197e-06, Test Score: 2.3604950758742694e-06, Time Cost: 0.010296821594238281
Epoch: 600, Test Loss: 3.379404006409459e-06, Test Score: 3.3794040429685477e-06, Time Cost: 0.01204991340637207
Epoch: 700, Test Loss: 2.7966346124230767e-06, Test Score: 2.7966346385678853e-06, Time Cost: 0.01173090934753418
Epoch: 800, Test Loss: 5.21520150869037e-06, Test Score: 5.2152016748364675e-0

In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 3.823421743615607e-07


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
